# Percent of Total Payroll
This notebook is for finding what % of total payroll some of our agencies make up in 2019

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sodapy import Socrata
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
client.timeout = 40
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
total_payroll_2019 = pd.DataFrame(client.get("rxhc-k6iz", where = "Year = 2019", limit = 999999))

In [4]:
total_payroll_2019.head(5)

year name_last   name_first  \
0  2019   LeBlanc     Kathleen   
1  2019    Horton  Christopher   
2  2019  Williams       Jalisa   
3  2019    DeLuca         Ross   
4  2019  Monaghan      Brendan   

                                 department_division  \
0  DEPARTMENT OF ELEMENTARY & SECONDARY EDUCATION...   
1                      STATE RECLAMATION BOARD (SRB)   
2                   WESTFIELD STATE UNIVERSITY (WSC)   
3                 BRIDGEWATER STATE UNIVERSITY (BSC)   
4   Massachusetts Bay Transportation Authority (MBT)   

             position_title         position_type         service_end_date  \
0      Program Coordinators  Part Time Contractor  2019-12-21T00:00:00.000   
1            Superintendent    Full Time Employee  2019-12-21T00:00:00.000   
2  Associate Vice President    Full Time Employee  2019-12-21T00:00:00.000   
3         PT Secondary Work  Part Time Contractor  2019-10-26T00:00:00.000   
4    Inspector,Construction    Full Time Employee  2019-12-21T00:00:00.000   

  pay_total_actual pay_base_actual pay_buyout_actual pay_overtime_actual  \
0         45716.63        45716.63                 0                   0   
1         58250.81        58250.81                 0                   0   
2         78273.21        78273.21                 0                   0   
3              400             400                 0                   0   
4        195054.26        95534.88                 0            99519.38   

  pay_other_actual annual_rate pay_year_to_date department_location_zip_code  \
0                0     56323.8         45716.63                        02148   
1                0       60450         58250.81                    012010000   
2                0      150858         78273.21                    010861630   
3                0        1300              400                    023250000   
4                0    100630.4        195054.26                        02116   

                               contract bargaining_group_no  \
0                        No Union (NON)                 NON   
1                        No Union (NON)                 NON   
2  Non-Unit Professionals (Hi Ed) (NUP)                 NUP   
3                        No Union (NON)                 NON   
4   MBTA L105 Technical Engineers (M65)                 M65   

           bargaining_group_title  trans_no chris  
0                        No Union  14609803   DOE  
1                        No Union  17905641   SRB  
2  Non-Unit Professionals (Hi Ed)  19202131   WSC  
3                        No Union  17284176   BSC  
4   MBTA L105 Technical Engineers   2212119   MBT

In [26]:
total_payroll_2019.loc[:,"pay_total_actual"] = total_payroll_2019.loc[:,"pay_total_actual"].astype(float)

In [27]:
total_pay = total_payroll_2019["pay_total_actual"].sum()

In [30]:
pct_by_agency = (total_payroll_2019.groupby("department_division").sum()["pay_total_actual"]\
                 .sort_values(ascending=False))/total_pay

In [32]:
pd.set_option("display.max.rows", 153)
pct_by_agency

department_division
UNIVERSITY OF MASSACHUSETTS SYSTEM (UMS)                      0.18243
Massachusetts Bay Transportation Authority (MBT)              0.07305
TRIAL COURT (TRC)                                             0.06568
DEPARTMENT OF CORRECTION (DOC)                                0.05472
DEPARTMENT OF STATE POLICE (POL)                              0.04567
DEPARTMENT OF DEVELOPMENTAL SERVICES (DMR)                    0.04260
DEPARTMENT OF CHILDREN AND FAMILIES (DSS)                     0.03894
MASSACHUSETTS DEPARTMENT OF TRANSPORTATION (DOT)              0.03853
DEPARTMENT OF MENTAL HEALTH (DMH)                             0.03062
DEPARTMENT OF PUBLIC HEALTH (DPH)                             0.02958
EXECUTIVE OFFICE OF HEALTH and HUMAN SERVICES (EHS)           0.01706
DEPARTMENT OF REVENUE (DOR)                                   0.01535
BRIDGEWATER STATE UNIVERSITY (BSC)                            0.01327
DEPARTMENT OF TRANSITIONAL ASSISTANCE (WEL)                   0.01312


In [33]:
benefits_estimate = 2717337093.050 + 32839480.940

In [34]:
pct_by_agency * benefits_estimate

department_division
UNIVERSITY OF MASSACHUSETTS SYSTEM (UMS)                      501711378.37387
Massachusetts Bay Transportation Authority (MBT)              200896161.34600
TRIAL COURT (TRC)                                             180641410.92484
DEPARTMENT OF CORRECTION (DOC)                                150480783.37483
DEPARTMENT OF STATE POLICE (POL)                              125605915.35126
DEPARTMENT OF DEVELOPMENTAL SERVICES (DMR)                    117168574.89707
DEPARTMENT OF CHILDREN AND FAMILIES (DSS)                     107084683.53287
MASSACHUSETTS DEPARTMENT OF TRANSPORTATION (DOT)              105977631.62215
DEPARTMENT OF MENTAL HEALTH (DMH)                              84199537.35977
DEPARTMENT OF PUBLIC HEALTH (DPH)                              81340495.03358
EXECUTIVE OFFICE OF HEALTH and HUMAN SERVICES (EHS)            46930713.29398
DEPARTMENT OF REVENUE (DOR)                                    42227168.22219
BRIDGEWATER STATE UNIVERSITY (BSC)          

Seems like a lot of money to add which is good. I'm pretty sure there are unemployment payments somewhere out there but I can't find them .

(D12) UNEMPLOYMENT COMPENSATION INSURANCE PREMIUM PAYMENT has 18 mil of spending, not very much

The department EXECUTIVE OFFICE OF LABOR AND WORKFORCE DEVELOPMENT (EOL) makes payments under object code R17 that are unemployment benefits but that is to all people in MA not just state employees. Maybe it's broken down by state gov or agency somewhere though

In [41]:
EOL = pd.DataFrame(client.get("pegc-naaa", 
                   where = "Department = 'EXECUTIVE OFFICE OF LABOR AND WORKFORCE DEVELOPMENT (EOL)' " 
                                       "AND budget_fiscal_year = 2019", limit = 999999))

In [50]:
EOL.loc[:,"amount"] = EOL.loc[:,"amount"].astype(float)

In [51]:
EOL.groupby("object_code").sum()["amount"].sort_values(ascending=False)

object_code
(R17) UNEMPLOYMENT BENEFITS                                          3772081578.62000
(A01) SALARIES: INCLUSIVE                                              77830186.97000
(PP1) GRANTS TO NON-PUBLIC ENTITIES                                    75553940.89000
(P01) GRANTS TO PUBLIC ENTITIES                                        18719707.34000
(T04) PAYMENTS & REFUNDS                                               14858947.13000
(U11) INFORMATION TECHNOLOGY (IT) CONTRACT SERVICES                     5357149.15000
(G01) SPACE RENTAL                                                      3256587.08000
(U10) INFORMATION TECH (IT) EQUIPMENT MAINTENANCE & REPAIR              2618193.84000
(U05) INFORMATION TECHNOLOGY (IT) TEMP STAFF AUGMENTATION PROFS         2143889.09000
(JJ1) LEGAL SUPPORT SERVICES                                            1943977.78000
(E06) POSTAGE                                                           1122208.12000
(HH3) MEDIA DESIGN, EDITORIAL AND COMMUNIC

In [48]:
EOL_unemployment_benefits = EOL[EOL["object_code"] == "(R17) UNEMPLOYMENT BENEFITS"]

In [49]:
EOL_unemployment_benefits["amount"].sum()

3772081578.6200004

In [1]:
total_pay

NameError: name 'total_pay' is not defined

In [8]:
total_payroll_2019[total_payroll_2019["department_division"].str.contains("CJT")]

year   name_last   name_first  \
12318   2019    Richmond  Christopher   
12333   2019       Lembo       Thomas   
12436   2019      Calter         Ryan   
12484   2019         Lee        Brian   
12578   2019   Fernandez     Cristina   
...      ...         ...          ...   
125733  2019     Jenkins      Stephen   
126233  2019    Williams         Mark   
126949  2019  Waczkowski     Jennifer   
127658  2019      Landry      Douglas   
127946  2019         May       Daniel   

                              department_division  \
12318   Municipal Police Training Committee (CJT)   
12333   Municipal Police Training Committee (CJT)   
12436   Municipal Police Training Committee (CJT)   
12484   Municipal Police Training Committee (CJT)   
12578   Municipal Police Training Committee (CJT)   
...                                           ...   
125733  Municipal Police Training Committee (CJT)   
126233  Municipal Police Training Committee (CJT)   
126949  Municipal Police Training Committee (CJT)   
127658  Municipal Police Training Committee (CJT)   
127946  Municipal Police Training Committee (CJT)   

                        position_title         position_type  \
12318   Instructors/Lecturers/Trainers  Full Time Contractor   
12333   Instructors/Lecturers/Trainers  Full Time Contractor   
12436   Instructors/Lecturers/Trainers  Full Time Contractor   
12484   Instructors/Lecturers/Trainers  Full Time Contractor   
12578   Instructors/Lecturers/Trainers  Full Time Contractor   
...                                ...                   ...   
125733  Instructors/Lecturers/Trainers  Full Time Contractor   
126233  Instructors/Lecturers/Trainers  Full Time Contractor   
126949  Instructors/Lecturers/Trainers  Full Time Contractor   
127658  Instructors/Lecturers/Trainers  Full Time Contractor   
127946       Regional Academy Director    Full Time Employee   

               service_end_date pay_total_actual pay_base_actual  \
12318   2019-06-08T00:00:00.000             1080            1080   
12333   2019-07-20T00:00:00.000             1760            1760   
12436   2019-12-07T00:00:00.000             1660            1660   
12484   2019-12-07T00:00:00.000             5860            5860   
12578   2019-12-07T00:00:00.000             1200            1200   
...                         ...              ...             ...   
125733  2019-08-03T00:00:00.000             1000            1000   
126233  2019-08-03T00:00:00.000             4600            4600   
126949  2019-08-03T00:00:00.000              640             640   
127658  2019-08-03T00:00:00.000              960             960   
127946  2019-04-13T00:00:00.000         33123.21         26859.8   

       pay_buyout_actual pay_overtime_actual pay_other_actual annual_rate  \
12318                  0                   0                0       83200   
12333                  0                   0                0       83200   
12436                  0                   0                0      104000   
12484                  0                   0                0      104000   
12578                  0                   0                0      104000   
...                  ...                 ...              ...         ...   
125733                 0                   0                0       83200   
126233                 0                   0                0       83200   
126949                 0                   0                0       83200   
127658                 0                   0                0       83200   
127946           6263.41                   0                0   90281.613   

       pay_year_to_date department_location_zip_code               contract  \
12318              1080                        02368         No Union (NON)   
12333              1760                        02368         No Union (NON)   
12436              1660                        02368         No Union (NON)   
12484              5860                        0236

In [9]:
cjt_payroll_2019 = pd.DataFrame(client.get("rxhc-k6iz", 
                                           where = "department_division = 'Municipal Police Training Committee (CJT)' "
                                           "AND Year = 2019", limit = 5))

In [10]:
cjt_payroll_2019

year  name_last   name_first                        department_division  \
0  2019   Richmond  Christopher  Municipal Police Training Committee (CJT)   
1  2019      Lembo       Thomas  Municipal Police Training Committee (CJT)   
2  2019     Calter         Ryan  Municipal Police Training Committee (CJT)   
3  2019        Lee        Brian  Municipal Police Training Committee (CJT)   
4  2019  Fernandez     Cristina  Municipal Police Training Committee (CJT)   

                   position_title         position_type  \
0  Instructors/Lecturers/Trainers  Full Time Contractor   
1  Instructors/Lecturers/Trainers  Full Time Contractor   
2  Instructors/Lecturers/Trainers  Full Time Contractor   
3  Instructors/Lecturers/Trainers  Full Time Contractor   
4  Instructors/Lecturers/Trainers  Full Time Contractor   

          service_end_date pay_total_actual pay_base_actual pay_buyout_actual  \
0  2019-06-08T00:00:00.000             1080            1080                 0   
1  2019-07-20T00:00:00.000             1760            1760                 0   
2  2019-12-07T00:00:00.000             1660            1660                 0   
3  2019-12-07T00:00:00.000             5860            5860                 0   
4  2019-12-07T00:00:00.000             1200            1200                 0   

  pay_overtime_actual pay_other_actual annual_rate pay_year_to_date  \
0                   0                0       83200             1080   
1                   0                0       83200             1760   
2                   0                0      104000             1660   
3                   0                0      104000             5860   
4                   0                0      104000             1200   

  department_location_zip_code        contract bargaining_group_no  \
0                        02368  No Union (NON)                 NON   
1                        02368  No Union (NON)                 NON   
2                        02368  No Union (NON)                 NON   
3                        02368  No Union (NON)                 NON   
4                        02368  No Union (NON)                 NON   

  bargaining_group_title  trans_no chris  
0               No Union  17167423   CJT  
1               No Union  17772599   CJT  
2               No Union  18192291   CJT  
3               No Union  14767220   CJT  
4               No Union  19585306   CJT